In [1]:
train_data_dir = "../dataset/018.감성대화/Training_221115_add/감성대화말뭉치(최종데이터)_Training.json"
val_data_dir = "../dataset/018.감성대화/Validation_221115_add/감성대화말뭉치(최종데이터)_Validation.json"

In [2]:
import pandas as pd

train_data = pd.read_json(train_data_dir)
val_data = pd.read_json(val_data_dir)

train_data.describe()

,profile,talk
count,51628,51628
unique,1919,51628
top,"{'persona-id': 'Pro_05368', 'persona': {'perso...","{'id': {'profile-id': 'Pro_05349', 'talk-id': ..."
freq,97,1


In [8]:
sentiments = """분노 슬픔 불안 상처 당황 기쁨
툴툴대는 실망한 두려운 질투하는 고립된 감사하는
좌절한 비통한 스트레스_받는 배신당한 남의_시선_의식하는 사랑하는
짜증나는 후회되는 취약한 고립된 외로운 편안한
방어적인 우울한 혼란스러운 충격_받은 열등감 만족스러운
악의적인 마비된 당혹스러운 불우한 죄책감 흥분되는
안달하는 염세적인 회의적인 희생된 부끄러운 느긋한
구역질_나는 눈물이_나는 걱정스러운 억울한 혐오스러운 안도하는
노여워하는 낙담한 조심스러운 괴로워하는 한심한 신이_난
성가신 환멸을_느끼는 초조한 버려진 혼란스러운 자신하는"""

sentiments = sentiments.replace("\n", " ").split(" ")

print(len(sentiments))

labeltoindex = [[sentiments[i+6*j] for j in range(10)] for i in range(6)]

60


In [11]:
def stoi(label):
    for i, labels in enumerate(labeltoindex):
        if label in labels:
            return i
    return -1

def itos(index):
    return labeltoindex[index]

In [12]:
stoi("분노")

0

In [35]:
index = 120
train_data.talk[index], train_data.profile[index]

({'id': {'profile-id': 'Pro_03694', 'talk-id': 'Pro_03694_00004'},
  'content': {'HS01': '나 빼고 전부 결혼했어. 모임에 나갔더니 나만 혼자야.',
   'SS01': '혼자만 싱글이라 속상하시군요. 이럴 때 상황을 극복할 좋은 방법이 있을까요?',
   'HS02': '나도 빨리 좋은 여자를 만나고 싶어. 앞으로는 소개팅 거절하지 않고 만나볼 거야. 언젠가는 좋은 인연을 만날 수 있겠지.',
   'SS02': '소개팅하다 보면 좋은 인연을 만나게 되겠군요.',
   'HS03': '',
   'SS03': ''}},
 {'persona-id': 'Pro_03694',
  'persona': {'persona-id': 'A02_G01_C01',
   'human': ['A02', 'G01'],
   'computer': ['C01']},
  'emotion': {'emotion-id': 'S05_D02_E43',
   'type': 'E43',
   'situation': ['S05', 'D02']}})

In [22]:
with open("label.txt", "w") as f:
    f.writelines("\n".join([" ".join(labels) for labels in labeltoindex]))

In [26]:
def load_label():
    with open("label.txt", "r") as f:
        loaded = f.readlines() 
    processed = [l.replace("\n", "").split(" ") for l in loaded]
    return processed
        

load_label()

[['분노', '툴툴대는', '좌절한', '짜증나는', '방어적인', '악의적인', '안달하는', '구역질_나는', '노여워하는', '성가신'], ['슬픔', '실망한', '비통한', '후회되는', '우울한', '마비된', '염세적인', '눈물이_나는', '낙담한', '환멸을_느끼는'], ['불안', '두려운', '스트레스_받는', '취약한', '혼란스러운', '당혹스러운', '회의적인', '걱정스러운', '조심스러운', '초조한'], ['상처', '질투하는', '배신당한', '고립된', '충격_받은', '불우한', '희생된', '억울한', '괴로워하는', '버려진'], ['당황', '고립된', '남의_시선_의식하는', '외로운', '열등감', '죄책감', '부끄러운', '혐오스러운', '한심한', '혼란스러운'], ['기쁨', '감사하는', '사랑하는', '편안한', '만족스러운', '흥분되는', '느긋한', '안도하는', '신이_난', '자신하는']]


In [164]:
import torch
from torch.utils.data import Dataset, DataLoader
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

class SentimentDataset(Dataset):
    def __init__(self, data, label, max_token_len):
        self.x = data
        self.y = label
        self.max_token_len = max_token_len
        
    def __getitem__(self, index):
        x = self.x[index]
        x = tokenizer.encode(x)
        if len(x) > self.max_token_len:
            x = x[:self.max_token_len]
        x = torch.tensor(x, device=torch.device("cuda"))
        x = torch.nn.functional.pad(x ,pad=(1, self.max_token_len-x.shape[0]-1),value = 1)
        y = self.y[index]
        y = torch.tensor(y, device=torch.device("cuda"))
        return x, y

    def __len__(self): return len(self.x)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [145]:
import json

texts = []
labels = []

for i, (profile, talk) in enumerate(zip(train_data.profile, train_data.talk)):
    text = list(talk["content"].values())[0]
    label = int(profile["emotion"]["type"][1]) - 1
    
    texts.append(text)
    labels.append(label)

In [146]:
labels[0]

0

In [165]:
max_token_len = 50
dataset = SentimentDataset(texts, labels, max_token_len=max_token_len)

In [166]:
dataset[0]

(tensor([   1,    2, 3803, 7086, 3466, 5002, 5002, 1363, 7096, 3282, 5591,  258,
         5112, 5330, 1406, 5782,   54,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1], device='cuda:0'),
 tensor(0, device='cuda:0'))

In [167]:
train_data_loader = DataLoader(dataset, batch_size=128, shuffle=True)

In [205]:
import torch
import torch.optim as optim
import tqdm

def train(model,train_data):
    model.train()

    optimizer = optim.AdamW(model.parameters(),lr = 0.005)
    loss_fn = torch.nn.CrossEntropyLoss()
   
    epochs = 30

    for epoch in range(epochs):
        t_loss = 0
        for x,y in tqdm.tqdm(train_data):        
            out = model(x)
            loss = loss_fn(out,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            t_loss += loss.detach().item()

        print(f"Epoch : {epoch + 1} / {epochs} \t Train  Loss : {t_loss/len(train_data) : .3f}")

    torch.save(model.state_dict(), "model.pt")

In [206]:
from model import SentimentAnalysis

model = SentimentAnalysis(tokenizer.vocab_size, 50, n_filters=5, filter_sizes=[2, 3, 4], pool_size=1, dropout=0.2, hidden_size=100, num_classes=6).to("cuda")

train(model, train_data_loader)

100%|██████████| 404/404 [00:22<00:00, 17.95it/s]


Epoch : 1 / 30 	 Train  Loss :  1.638


100%|██████████| 404/404 [00:23<00:00, 17.54it/s]


Epoch : 2 / 30 	 Train  Loss :  1.511


100%|██████████| 404/404 [00:22<00:00, 18.02it/s]


...

Epoch : 28 / 30 	 Train  Loss :  1.338


100%|██████████| 404/404 [00:22<00:00, 17.59it/s]


Epoch : 29 / 30 	 Train  Loss :  1.336


100%|██████████| 404/404 [00:22<00:00, 17.98it/s]

Epoch : 30 / 30 	 Train  Loss :  1.334


In [193]:
def generate(command):
    x = tokenizer.encode(command)
    if len(x) > max_token_len:
        x = x[:max_token_len]
    x = torch.tensor(x, device=torch.device("cuda"))
    x = torch.nn.functional.pad(x ,pad=(1, max_token_len-x.shape[0]-1),value = 1)
    x = x.unsqueeze(0)
    
    pred = model(x)
    
    return torch.argmax(pred, dim=1)[0].detach().cpu()

In [198]:
itos(generate("안녕안녕 반가워!").tolist())

['기쁨', '감사하는', '사랑하는', '편안한', '만족스러운', '흥분되는', '느긋한', '안도하는', '신이_난', '자신하는']

In [200]:
itos(generate("이제 직접 실행시켜서 확인해봅시다!").tolist())

['기쁨', '감사하는', '사랑하는', '편안한', '만족스러운', '흥분되는', '느긋한', '안도하는', '신이_난', '자신하는']

In [178]:
itos(2)

['불안',
 '두려운',
 '스트레스_받는',
 '취약한',
 '혼란스러운',
 '당혹스러운',
 '회의적인',
 '걱정스러운',
 '조심스러운',
 '초조한']